In [77]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import os
from urllib2 import urlopen
import urllib
import requests
from pymongo import MongoClient
import json#for storing beautiful soup to mongo

In [78]:
client = MongoClient()

In [79]:
img_db = client['images_final']
image_data = img_db['image_data']

In [80]:
#To get a beautiful soup object
def make_soup(url):
    
    #html = urlopen(url).read()
    page = requests.get(url)
    html = page.text
    return BeautifulSoup(html, 'html.parser')


In [105]:
#Returns the product specification table FROM amazon product page
def get_prod_table(soup):
    tab = soup.select('table.a-keyvalue.a-spacing-mini')[0] # Because it returns 1 table in a list
    #print tab
    table = {} 
    for tr in tab.findAll('tr'):
        try: 
            table[str(tr.th.get_text().strip())]= str((tr.td.get_text()).strip())
        except:
            #return "Not available"#No exception raised here since not needed. We can move to next element of table
            pass #will ensure that this image and its metadata are not returned moving onto the next link
    return table   

In [97]:
#Returns the product description FROM amazon product page
def get_prod_desc(soup):
    try :
        desc = soup.findAll('div',attrs={'id' : 'productDescription'})[0]#all product description are kept like this
        return desc.get_text().strip()
    except:
        return "Not available" #Returns not available when no description

In [98]:
#Returns the title of the amazon product page
def get_prod_title(soup):
    try:
        title = soup.findAll('span',attrs={'id' : 'productTitle'})[0]
        return str(title.get_text().strip())  
    except:
        return "Not available" #Returns not available when no title(generaly never the case)

In [99]:
#Insert data into mongodb

def insert_to_mongo(soup,prod_url,img_url,link_counter):
    
    #Get decription table and product description
    table = get_prod_table(soup)
    
    description = get_prod_desc(soup)
    title = get_prod_title(soup)
    try:
        image_data.insert_one({ 'img_no' : link_counter,'prod_url':prod_url,'img_url': img_url,"prod_info_table":table,"prod_desc": description, "title" :title  })
    except:
        print "MongoDb Insertion Error %d"%link_counter

In [100]:
# Given a link and the number of link creates a folder using number and stores images in it   
#Link counter counts each page images are scraped from used to save the images
def get_images(prod_url, link_counter):  
    
    #Make soup with product url
    soup = make_soup(prod_url)
    
    #this makes a list of bs4 element image tags
    images = [img for img in soup.select('img.a-dynamic-image.a-stretch-vertical')]
    
    #print (str(len(images)) + "images found.")
    #print 'Downloading images to current working directory.'
    #compile our unicode list of image links
    
    image_links = [each['data-old-hires'] for each in images]
    
    #print image_links
    #To standardize the name of each image on prod_url we use file_name as a counter
    file_name = 0
    
    #Create a sub directory for each link and go to that directory
    os.mkdir('{0}'.format(link_counter))
    os.chdir('{0}'.format(link_counter)) #Decided to keep all data in one directory
    
    
    
    #Get images and store them 
    for each in image_links:
        try:
            filename='{0}.jpg'.format(link_counter)
            #print "???????????????", filename
            #print each
            insert_to_mongo(soup,prod_url,each,link_counter)
            
            urllib.urlretrieve(each, filename)
            #Alternative to urlretrieve
            #r = requests.get(each)
            #with open(filename,'wb') as f:
            #f.write(r.content)
            file_name  += 1
            #insert text to mongodb
            
            
        except:
            pass
            #print "Image Insertion Error %d"%link_counter #Insertion error will automatically be raised from insert_to_mongo
        
    # return to original directory
    os.chdir('..')
    return image_links

In [101]:

def lets_scrape_amazon(init_url,no_of_pages, prog_track_interval = 10):

    link_counter  = 0

    for i in range(1,(no_of_pages+1)):
        # Link for wristwatches with rating > 4 stars
        url =init_url.format(i,i)
        
        #Make soup with the search page
        soup = make_soup(url)
        
        #Get the individual links to the product for each search page
        links = [link.get('href') for link in soup.select("a.a-link-normal.s-access-detail-page")]
        
        for link in links:
            get_images(link,link_counter)
            #Printing to track progress
            if (link_counter% prog_track_interval) == 0 :
                print "%d images downloaded"%(link_counter)
            link_counter += 1 
        

In [102]:
watch_search_url = "http://www.amazon.com/s/ref=sr_pg_{0}?fst=as%3Aoff&rh=n%3A7141123011%2Cn%3A10445813011%2Cn%3A7147441011%2Cn%3A6358539011%2Cn%3A6358540011%2Ck%3Awrist+watches%2Cp_72%3A2661618011&page={1}&bbn=10445813011&keywords=wrist+watches&ie=UTF8&qid=1451726453"

In [106]:
lets_scrape_amazon(watch_search_url,100,100)

0 images downloaded
100 images downloaded
200 images downloaded
300 images downloaded
400 images downloaded
500 images downloaded
600 images downloaded
700 images downloaded
800 images downloaded
900 images downloaded
1000 images downloaded
1100 images downloaded
1200 images downloaded
1300 images downloaded
1400 images downloaded
1500 images downloaded
1600 images downloaded
1700 images downloaded
1800 images downloaded
1900 images downloaded
2000 images downloaded
2100 images downloaded
2200 images downloaded
2300 images downloaded
2400 images downloaded
2500 images downloaded
2600 images downloaded
2700 images downloaded
2800 images downloaded
2900 images downloaded
3000 images downloaded
3100 images downloaded
3200 images downloaded
3300 images downloaded
3400 images downloaded
3500 images downloaded
3600 images downloaded
3700 images downloaded
3800 images downloaded
3900 images downloaded
4000 images downloaded
4100 images downloaded
4200 images downloaded
4300 images downloaded


In [95]:
!pwd

/Users/Iskandar/Desktop/WatchSeer/Data2
